<a target="_parent" href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/data-designer/data-designer-101/2-structured-outputs-and-jinja-expressions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🎨 Data Designer 101: Structured Outputs and Jinja Expressions

In this notebook, we will continue our exploration of  `DataDesigner`, demonstrating more advanced data generation using structured outputs and Jinja expressions.


If this is your first time using `DataDesigner`, we recommend starting with the [first notebook](https://github.com/gretelai/gretel-blueprints/blob/main/docs/notebooks/data-designer/data-designer-101/1-the-basics.ipynb) in this 101 series.


<br>

### 💾 Install `gretel-client` and its dependencies

In [ ]:
%%capture
%pip install git+https://github.com/gretelai/gretel-python-client

In [ ]:
from gretel_client.navigator_client import Gretel

# We import AIDD column and parameter types using this shorthand for convenience.
import gretel_client.data_designer.params as P
import gretel_client.data_designer.columns as C

# The Gretel object is the SDK's main entry point for interacting with Gretel's API.
gretel = Gretel(api_key="prompt")

## 🧑‍🎨 Designing our data

- We will again create a product review dataset, but this time we will use structured outputs and Jinja expressions.

- Structured outputs lets you specify the exact schema of the data you want to generate. 

- `DataDesigner` supports schemas specified using either json schema or Pydantic data models (recommended).

<br>

We'll define our structured outputs using Pydantic data models:

In [ ]:
from decimal import Decimal
from typing import Literal
from pydantic import BaseModel, Field

# We define a Product schema so that the name, description, and price are generated 
# in one go, with the types and constraints specified.
class Product(BaseModel):
    name: str = Field(description="The name of the product")
    description: str = Field(description="A description of the product")
    price: Decimal = Field(description="The price of the product", ge=10, le=1000, decimal_places=2)

class ProductReview(BaseModel):
    rating: int = Field(description="The rating of the product", ge=1, le=5)
    customer_mood: Literal["irritated", "mad", "happy", "neutral", "excited"] = Field(description="The mood of the customer")
    review: str = Field(description="A review of the product")

Next, let's design our product review dataset using a few more tricks compared to the previous notebook:

In [ ]:
aidd = gretel.data_designer.new(model_suite="apache-2.0")

# Since we often just want a few attributes from Person objects, we can use 
# DataDesigner's `with_person_samplers` method to create multiple person samplers 
# at once and drop the person object columns from the final dataset.
aidd.with_person_samplers({"customer": P.PersonSamplerParams(age_range=[18, 65])})

aidd.add_column(
    C.SamplerColumn(
        name="product_category", 
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
                values=["Electronics", "Clothing", "Home & Kitchen", "Books", "Home Office"], 
            )
    )
)

aidd.add_column(    
    C.SamplerColumn(
        name="product_subcategory",
        type=P.SamplerType.SUBCATEGORY,
        params=P.SubcategorySamplerParams(
            category="product_category",  
            values={
                "Electronics": ["Smartphones", "Laptops", "Headphones", "Cameras", "Accessories"],
                "Clothing": ["Men's Clothing", "Women's Clothing", "Winter Coats", "Activewear", "Accessories"],
                "Home & Kitchen": ["Appliances", "Cookware", "Furniture", "Decor", "Organization"],
                "Books": ["Fiction", "Non-Fiction", "Self-Help", "Textbooks", "Classics"],
                "Home Office": ["Desks", "Chairs", "Storage", "Office Supplies", "Lighting"]
            }
        )
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="target_age_range",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(values=["18-25", "25-35", "35-50", "50-65", "65+"])
    )
)

aidd.add_column(
    C.SamplerColumn(
        name="review_style",
        type=P.SamplerType.CATEGORY,
        params=P.CategorySamplerParams(
            values=["rambling", "brief", "detailed", "structured with bullet points"],
            weights=[1, 2, 2, 1]
        )
    )
)

# We can create new columns using Jinja expressions that reference 
# existing columns, including attributes of nested objects.
aidd.add_column(
    C.ExpressionColumn(
        name="customer_name",
        expr="{{ customer.first_name }} {{ customer.last_name }}"
    )
)

aidd.add_column(
    C.ExpressionColumn(
        name="customer_age",
        expr="{{ customer.age }}"
    )
)

# Add an `LLMStructuredColumn` column to generate structured outputs.
aidd.add_column(
    C.LLMStructuredColumn(
        name="product",
        prompt=(
            "Create a product in the '{{ product_category }}' category, focusing on products  "
            "related to '{{ product_subcategory }}'. The target age range of the ideal customer is "
            "{{ target_age_range }} years old. The product should be priced between $10 and $1000."
        ),
        output_format=Product
    )
)

aidd.add_column(
    C.LLMStructuredColumn(
        name="customer_review",
        prompt=(
            "Your task is to write a review for the following product:\n\n"
            "Product Name: {{ product.name }}\n"
            "Product Description: {{ product.description }}\n"
            "Price: {{ product.price }}\n\n"
            "Imagine your name is {{ customer_name }} and you are from {{ customer.city }}, {{ customer.state }}. "
            "Write the review in a style that is '{{ review_style }}'."
        ),
        output_format=ProductReview
    )
)

# Let's add an evaluation report to our dataset.
aidd.with_evaluation_report().validate()

## 👀 Preview the dataset

- Iteration is key to generating high-quality synthetic data.

- Use the `preview` method to generate 10 records for inspection.

- Setting `verbose_logging=True` prints logs within each task of the generation process.

In [ ]:
preview = aidd.preview(verbose_logging=True)

In [ ]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset.df.head()

In [ ]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

## 🆙 Scale up!

- Once you are happy with the preview, scale up to a larger dataset by submitting a batch workflow.

- Setting `wait_until_done=True` will block until the workflow is complete.

- You can view the evaluation report by following the workflow link in the output of `create` below.

In [ ]:
# This will take 5-10 minutes to complete.
workflow_run = aidd.create(
    num_records=100, 
    name="aidd-101-notebook-2-product-reviews",
    wait_until_done=True
)

In [ ]:
# The generated dataset is available as a pandas DataFrame.
workflow_run.dataset.df.head()

In [7]:
# Fetch the evaluation report from the workflow run.
report = workflow_run.report

# If running in colab:
report.display_in_notebook()

# If running locally, we recommend displaying in the browser.
#report.display_in_browser()